In [2]:
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import pytz
import openpyxl

#paste with ctrl+shift+v into calendar and format columns as text in web version as there is no format for snow


def get_date_range(file_path, start_row, end_row, target_timezone):

    # Load the workbook 
    workbook = openpyxl.load_workbook(file_path)

    # Select the sheet (active by default, modify if you need another month sheet)
    sheet = workbook.active
    
    # Set column indices for the Specified columns
    collection_col = get_column_index(sheet, "Collection")
    description_col = get_column_index(sheet, "Week and Day")
    start_time_col = get_column_index(sheet, "Start Time")
    time_zone_col = get_column_index(sheet, "Time Zone")
    duration_col = get_column_index(sheet, "Duration")
    Start_Date_col = get_column_index(sheet, "Start Date")
    End_Date_col = get_column_index(sheet, "End Date")

    for row_num in range(start_row, end_row + 1):
        
        # Skip rows containing the specific text
        if sheet.cell(row=row_num, column=2).value == "CRs created/left:":
            continue

        # Get data from specified columns in the current row
        collection = sheet.cell(row=row_num, column=collection_col).value
        print(f"Collection name {collection}")
        start_description = sheet.cell(row=row_num, column=description_col).value
        print(f"Start Description: {start_description}")
        start_time = sheet.cell(row=row_num, column=start_time_col).value
        print(f"Start Time: {start_time}")
        start_timezone = sheet.cell(row=row_num, column=time_zone_col).value
        print(f"Start Timezone: {start_timezone}")
        duration_hours = sheet.cell(row=row_num, column=duration_col).value
        duration_hours = int(duration_hours.replace(" hours", ""))
        print(f"Duration (hours): {duration_hours}")
        
        # Call the converter function to get the start_datetime_target and end_datetime_target
        start_datetime_string, end_datetime_string = converter(start_description, start_time, start_timezone, duration_hours, target_timezone, collection)

        # Write the values to the "Start Date" and "End Date"
        sheet.cell(row=row_num, column=Start_Date_col).value = start_datetime_string
        sheet.cell(row=row_num, column=End_Date_col).value = end_datetime_string
    
    # Save the opened sheet
    workbook.save(output_path)

    print(f"Converted to {target_timezone} date filled in '{output_path}'")
        
def get_column_index(sheet, column_name):
    # Get the column index based on the column name
    for col_num in range(1, sheet.max_column + 1):
        if sheet.cell(row=1, column=col_num).value == column_name:
            return col_num
    raise ValueError(f"Column '{column_name}' not found in the sheet")

def get_pytz_timezone(start_timezone, collection):
    # Mapping friendly time zone names to pytz time zone names
        timezone_mapping = {
        'argentina': 'America/Argentina/Buenos_Aires',
        'pst': 'America/Los_Angeles',
        'pdt': 'America/Los_Angeles',
        'pst/pdt': 'America/Los_Angeles',
        'PST': 'America/Los_Angeles',
        'PST/PDT': 'America/Los_Angeles',
        'CT(US&Canada)': 'US/Central',
        'Central Time(US)': 'US/Central',
        'ET(US&Canada)': 'US/Eastern',
        'CET': 'Europe/Berlin',
        'CET/CEST': 'Europe/Berlin',
        'cet/cest': 'Europe/Berlin',
        'cet': 'Europe/Berlin',
        'chile': 'America/Santiago',
        'bst/gmt': 'Europe/London',
        'bst': 'Europe/London',
        'BST': 'Europe/London',
        'GMT': 'Europe/London',
        'UTC': 'Etc/UTC',
        'BST/GMT': 'Europe/London',
        'colombia': 'America/Bogota',
        'Central Time(US)': 'US/Central',
        'AEST': 'Australia/Sydney',
        'aest': 'Australia/Sydney',
        'NZDT': 'NZ',
        'JST': 'Japan',
        'Brazil': 'America/Noronha',
        'Brussels': 'Europe/Brussels',
        'Dublin': 'Europe/Dublin',
        'France': 'Europe/Paris',
        'India': 'Asia/Kolkata',
        'Argentina': 'America/Argentina/Buenos_Aires',
        'Colombia': 'America/Bogota',
        'Mexico': 'America/Mexico_City',
        'Spain': 'Europe/Madrid',
        'China': 'Asia/Shanghai',
        'Greece': 'Europe/Athens',
        'Portugal': 'Europe/Lisbon',
        'Taiwan': 'Asia/Taipei',
        'UK': 'Europe/London',
        'Beijing': 'Hongkong',
        
        #add
        }

        if start_timezone.lower() == 'local time':
            # Split the collection name and look for location names
            for zone in collection.split():
                pytz_timezone = timezone_mapping.get(zone)
                if pytz_timezone:
                    print(f"mapped timezone: {pytz_timezone}")
                    return pytz_timezone
            
        else:
            pytz_timezone = timezone_mapping.get(start_timezone.strip())
            if pytz_timezone:
                print(f"Found timezone: {pytz_timezone}")
                return pytz_timezone
            else:
                raise ValueError(f"No timezone found for '{start_timezone}'")

def output(start_timezone, start_datetime, end_datetime, target_timezone, start_datetime_target, end_datetime_target):
        
        #test function to print vars
        print(f"\nOriginal Converted date to mapped {start_timezone} Start Time Zone:")
        print(f"Start Date : {start_datetime.strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"End Date: {end_datetime.strftime('%Y-%m-%d %H:%M:%S')}")
        print("")

        print(f"Converted to {target_timezone} Target Time Zone:")
        print(f"Start Date : {start_datetime_target.strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"End Date: {end_datetime_target.strftime('%Y-%m-%d %H:%M:%S')}")
        print("")

        return output

def converter(start_description, start_time, start_timezone, duration_hours, target_timezone, collection):
        
    day_names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    day_indices = {day: index for index, day in enumerate(day_names)}
    description_parts = start_description.split()

    start_datetime = None
    end_datetime = None

    # Look for correct format of the start_time within string where comma split description by parts
    if ',' in ' '.join(start_time):

        time_parts = start_time.split(', ')
        specified_time = time_parts[0].strip() 
        specified_day = time_parts[1].strip() #added eemoving of spaces

    elif any(day in description_parts for day in day_names): # look for weekday in start_description

        specified_day = next(day for day in day_names if day in description_parts)
        specified_time = start_time.strip()

    elif 'Last Day'.lower() in ' '.join(description_parts).lower():

        specified_day = None
        specified_time = start_time.strip() #added eemoving of spaces

    else:

        raise ValueError(f"Column '{start_time}' has wrong format")

    # find current date
    if '.' in specified_time:
        start_time_24h = datetime.strptime(specified_time, "%I.%M%p").strftime("%H:%M")
    else:
        start_time_24h = datetime.strptime(specified_time, "%I%p").strftime("%H:%M")

    current_date = datetime.now()

    # Convert start_timezone to pytz time zone
    start_timezone_pytz = get_pytz_timezone(start_timezone, collection)

    start_timezone = start_timezone_pytz

    # Finding the next Monday after the first Tuesday (Patch Tuesday)
    first_day_of_month = current_date.replace(day=1)
    first_tuesday = first_day_of_month + timedelta(days=(1 - first_day_of_month.weekday()) % 7)
    patch_tuesday = first_tuesday + timedelta(days=6)
    next_monday = patch_tuesday + timedelta(days=(7 - patch_tuesday.weekday()) % 7)


    if 'next'.lower() in ' '.join(description_parts).lower():

        print("Executing 'Next Month' logic...")
        # Extracting the month
        current_date = current_date.replace(day=1) + relativedelta(months=1)

        # Extracting numeric value for weekday selection (1st, 2nd, 3rd, 4th)
        weekday_selection = int(start_description.split()[0][:-len("th")])

        # Calculating the selected weekday of the next month
        first_day_of_next_month = current_date.replace(day=1)
        selected_weekday = day_indices[specified_day.capitalize()]
        current_weekday = first_day_of_next_month.weekday()
        diff = (selected_weekday - current_weekday) % 7
        start_date = first_day_of_next_month + timedelta(days=diff + 7 * (weekday_selection - 1))

        # Constructing start date and time
        start_datetime = datetime.strptime(f"{start_date.date()} {start_time_24h}", "%Y-%m-%d %H:%M")
        start_datetime = pytz.timezone(start_timezone_pytz).localize(start_datetime)
        end_datetime = start_datetime + timedelta(hours=duration_hours)

        # Convert to the specified target time zone
        start_datetime_target = start_datetime.astimezone(pytz.timezone(target_timezone))
        end_datetime_target = end_datetime.astimezone(pytz.timezone(target_timezone))

        output(start_timezone, start_datetime, end_datetime, target_timezone, start_datetime_target, end_datetime_target)

        return start_datetime_target.strftime("%Y-%m-%d %H:%M:%S"), end_datetime_target.strftime("%Y-%m-%d %H:%M:%S")

    elif 'after'.lower() in ' '.join(description_parts).lower():

        print("Executing 'after Patch Tuesday' logic...")
        
        if description_parts[0] == '0':
            start_day_index = day_indices[specified_day.capitalize()] - next_monday.weekday()
        else:
            # Extracting the week offset
            week_offset = int(description_parts[0][:-2])
            # Calculate the starting day index within the desired week after Patch Tuesday
            start_day_index = (day_indices[specified_day.capitalize()] - next_monday.weekday() + 7) % 7
            # Adjust to the specified week within the month
            start_day_index += week_offset * 7
            # Adjust if the start_day_index is negative
            if start_day_index < 0:
                start_day_index += 7

        # Constructing start date and time
        start_date = next_monday + timedelta(days=start_day_index)
        start_datetime = datetime.strptime(f"{start_date.date()} {start_time_24h}", "%Y-%m-%d %H:%M")
        start_datetime = pytz.timezone(start_timezone_pytz).localize(start_datetime)
        end_datetime = start_datetime + timedelta(hours=duration_hours)

        # Convert to the specified target time zone
        start_datetime_target = start_datetime.astimezone(pytz.timezone(target_timezone))
        end_datetime_target = end_datetime.astimezone(pytz.timezone(target_timezone))

        output(start_timezone, start_datetime, end_datetime, target_timezone, start_datetime_target, end_datetime_target)

        return start_datetime_target.strftime("%Y-%m-%d %H:%M:%S"), end_datetime_target.strftime("%Y-%m-%d %H:%M:%S")

    elif 'of the month'.lower() in ' '.join(description_parts).lower():

        print("Executing 'of the' logic...")
        # Extracting numeric value for weekday selection (1st, 2nd, 3rd, 4th)
        weekday_selection = int(description_parts[0][:-len("th")])

        # Finding the current month's first day
        first_day_of_month = current_date.replace(day=1)

        # Finding the desired weekday of the month
        selected_weekday = day_indices[specified_day.capitalize()]
        current_weekday = first_day_of_month.weekday()

        # Calculate the starting day index within the adjusted month
        diff = (selected_weekday - current_weekday + 7) % 7
        start_date = first_day_of_month + timedelta(days=diff + 7 * (weekday_selection - 1))

        # Check if the calculated date is beyond the end of the month
        if start_date.month != first_day_of_month.month:
            start_date -= timedelta(days=7)

        # Adjusting for leap year if the selected month is February
        if start_date.month == 2 and start_date.day > 28:
            # Check if it's a leap year
            if (start_date.year % 4 == 0 and start_date.year % 100 != 0) or (start_date.year % 400 == 0):
                # Adjust to the last day of February in a leap year
                start_date = start_date.replace(day=29)

        # Constructing start date and time
        start_datetime = datetime.strptime(f"{start_date.date()} {start_time_24h}", "%Y-%m-%d %H:%M")
        start_datetime = pytz.timezone(start_timezone_pytz).localize(start_datetime)
        end_datetime = start_datetime + timedelta(hours=duration_hours)

        # Convert to the specified target time zone
        start_datetime_target = start_datetime.astimezone(pytz.timezone(target_timezone))
        end_datetime_target = end_datetime.astimezone(pytz.timezone(target_timezone))

        output(start_timezone, start_datetime, end_datetime, target_timezone, start_datetime_target, end_datetime_target)

        return start_datetime_target.strftime("%Y-%m-%d %H:%M:%S"), end_datetime_target.strftime("%Y-%m-%d %H:%M:%S")

    elif 'last day'.lower() in ' '.join(description_parts).lower():

        print("Executing 'Last Day' logic...")
        # Constructing start date and time
        start_date = current_date.replace(day=1) + relativedelta(months=1, days=-1)
        start_datetime = datetime.strptime(f"{start_date.date()} {start_time_24h}", "%Y-%m-%d %H:%M")
        start_datetime = pytz.timezone(start_timezone_pytz).localize(start_datetime)
        end_datetime = start_datetime + timedelta(hours=duration_hours)

        # Convert to the specified target time zone
        start_datetime_target = start_datetime.astimezone(pytz.timezone(target_timezone))
        end_datetime_target = end_datetime.astimezone(pytz.timezone(target_timezone))

        output(start_timezone, start_datetime, end_datetime, target_timezone, start_datetime_target, end_datetime_target)

        return start_datetime_target.strftime("%Y-%m-%d %H:%M:%S"), end_datetime_target.strftime("%Y-%m-%d %H:%M:%S")

    else:
        # Additional cases of weekday logic...
        raise ValueError(f"Column '{start_description}' has wrong format")

if __name__ == "__main__":
    
    target_timezone = 'Europe/London'

    # Get input for file path and row numbers

    links_file_path = '../../../links.txt'

    with open(links_file_path, 'r', encoding='utf-8') as file:
    # Read the lines
        lines = file.readlines()

    file_path = lines[0].strip() if len(lines) > 0 else ''
    output_path = lines[1].strip() if len(lines) > 1 else ''
    
    start_row = int(input("Enter the start row: "))
    end_row = int(input("Enter the end row: "))
    
   
    #if file_path and row_num = 
    get_date_range(file_path, start_row, end_row, target_timezone)
    



Collection name AWS68 Bravado Webshop Germany PROD
Start Description: 1st week after Patch Tuesday, Monday
Start Time: 2AM
Start Timezone: CET/CEST
Duration (hours): 4
Found timezone: Europe/Berlin
Executing 'after Patch Tuesday' logic...

Original Converted date to mapped Europe/Berlin Start Time Zone:
Start Date : 2024-03-18 02:00:00
End Date: 2024-03-18 06:00:00

Converted to Europe/London Target Time Zone:
Start Date : 2024-03-18 01:00:00
End Date: 2024-03-18 05:00:00

Collection name AWS69 German IT services PROD
Start Description: 1st week after Patch Tuesday, Monday
Start Time: 2AM
Start Timezone: CET/CEST
Duration (hours): 4
Found timezone: Europe/Berlin
Executing 'after Patch Tuesday' logic...

Original Converted date to mapped Europe/Berlin Start Time Zone:
Start Date : 2024-03-18 02:00:00
End Date: 2024-03-18 06:00:00

Converted to Europe/London Target Time Zone:
Start Date : 2024-03-18 01:00:00
End Date: 2024-03-18 05:00:00

Collection name AWS38 Capitol Records DEV
Start D